In [3]:
import os
import traceback
import json
import pandas as pd
from dotenv import load_dotenv,find_dotenv

_:bool=load_dotenv(find_dotenv())

In [100]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
import os

llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro",google_api_key=os.getenv("GEMINIULTRA_API_KEY"),temperature=0.2)

response_format={
    "1":{
        "question":"question text",
        "options":{
            "a":"text for option",
            "b":"text for option",
            "c":"text for option",
            "d":"text for option"
        },
        "complexity":"complexity of the question",
        "answer":"correct option letter"
    }
}
response_format_json = json.dumps(response_format)

TEMPLATE = """
You are an expert MCQ maker, your job is to create {number} {question_type} type questions with {complexity} complexity on the topic "{topic}", once questions are created check each 
MCQ for mistakes and ensure that no question is repeated and every MCQ has a correct option and the correct answer, if you find any mistake, you will correct it, and then check again until there is no mistake.

Additional Instructions to follow : {instructions}.
Use this information to create the MCQ's : {context}

### Response format must be an object like this sample : {response_format}
"""
quiz_prompt=PromptTemplate(
    input_variables=["topic","number","question_type","complexity","context","instructions","response_format"],
    template=TEMPLATE
)
quiz_chain=LLMChain(llm=llm,prompt=quiz_prompt,verbose=True,output_key="quiz")
# mcqs=quiz_chain.run(topic="python", number=10, question_type="True False Unsure", complexity="medium", context="user your own knowledge", instructions="a question can have more than 1 answer,don't provide the answer with the questions, provide asnwers seperately", response_format=response_format)


REVIEW_TEMPLATE = """
You are an expert MCQ reviewer, your job is to review {number} {question_type} type questions with {complexity} complexity on the topic "{topic}", If you find any incorrect question or options or answer, update it properly, quiz:{quiz}. The output must be the same format as quiz input"""
review_prompt=PromptTemplate(input_variables=["topic","number","question_type","complexity","quiz"], template=REVIEW_TEMPLATE)
review_chain=LLMChain(llm=llm, prompt=review_prompt, verbose=True, output_key="review")


sequence_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["topic", "number", "question_type", "complexity","context","instructions","response_format"], output_variables=["review"] , verbose=True)
review=sequence_chain.run(topic="python", number=5, question_type="True False Unsure", complexity="medium", context="user your own knowledge", 
                   instructions="a question can have more than 1 answer", response_format=response_format_json)


review = review.strip('`')
j_format=json.loads(review)
data_list = []
for key, value in j_format.items():
    row = value
    row['options'] = json.dumps(value['options'])  # Convert options to a JSON string
    data_list.append(row)

quiz_dataframe:pd.DataFrame=pd.DataFrame(data_list)
print(quiz_dataframe)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert MCQ maker, your job is to create 5 True False Unsure type questions with medium complexity on the topic "python", once questions are created check each 
MCQ for mistakes and ensure that no question is repeated and every MCQ has a correct option and the correct answer, if you find any mistake, you will correct it, and then check again until there is no mistake.

Additional Instructions to follow : a question can have more than 1 answer.
Use this information to create the MCQ's : user your own knowledge

### Response format must be an object like this sample : {"1": {"question": "question text", "options": {"a": "text for option", "b": "text for option", "c": "text for option", "d": "text for option"}, "complexity": "complexity of the question", "answer": "correct option letter"}}


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are 